In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import yfinance as yf
from datetime import datetime, timedelta

### Data Download

In [127]:
# load data
# Download data from Yahoo Finance
tickers = ['AAPL', 'AMZN', 'TSLA', 'JNJ', 'COP', 'KO', 'JPM', 'NVDA' ,'UNH', 'PG']

end_date = datetime.now() - timedelta(days = 1)
start_date = end_date - timedelta(days = 5 * 365)

folder = 'AssignmentData'
if not os.path.exists(folder):
    os.mkdir(folder)

holder = {}
for ticker in tickers:

    file_name = f'{folder}/{ticker}.csv'
    if not os.path.exists(file_name):
        df = yf.Ticker(ticker).history(start = start_date.strftime("%Y-%m-%d"), end = end_date.strftime("%Y-%m-%d"))
        df.to_csv(file_name)

    df = pd.read_csv(file_name, index_col=0)
    df.index = pd.to_datetime(df.index, utc=True)
    df.index = df.index.tz_convert('US/Eastern')

    holder[ticker] = df

### Data Preprocessing

In [155]:
from sklearn.preprocessing import StandardScaler
log_returns = {}
for ticker, data in holder.items():
    log_return = np.log(data.Close / data.Close.shift(1)).dropna()
    standard_returns = log_return / log_return.rolling(252).std()
    log_returns[ticker] = {
                            "log_returns": log_return,
                            "std_returns": standard_returns.dropna()
                          }


### Filtered Historical Simulation

In [159]:
from arch import arch_model
conf_interval = 5
stock_vars = {}

for ticker, log_data in log_returns.items():
    log_return, std_return = log_data.values()
    model = arch_model(log_return * 100, vol = 'Garch', p = 1, q = 1)
    model_fit = model.fit(disp='off')
    omega, alpha, beta = model_fit.params[1:]
    next_day_std = (omega + alpha * log_return.iloc[-1]**2 + beta * std_return.iloc[-1]**2)**0.5
    
    # Randomly select 5000 standardized returns from the historical z-scores
    random_selection = np.random.choice(std_return, 10000)
    
    # Forecasts of the returns
    forecasts = next_day_std * random_selection

    var = np.percentile(forecasts, conf_interval)
    stock_vars[ticker] = var
    
    print(f"{ticker} VaR at {conf_interval}%: {var}%")



AAPL VaR at 5%: -0.8247630186530122%
AMZN VaR at 5%: -1.3725190695491867%
TSLA VaR at 5%: -2.0532099781859037%
JNJ VaR at 5%: -1.2384402495487101%
COP VaR at 5%: -0.4024215232570128%
KO VaR at 5%: -0.9977640184181414%
JPM VaR at 5%: -2.250844011709054%
NVDA VaR at 5%: -2.3099754167348943%
UNH VaR at 5%: -2.0352092406867985%
PG VaR at 5%: -2.1412577565449475%


### Cornish-Fisher Expansion

In [161]:
from scipy.stats import skew, kurtosis
cf_stock_vars = {}
for ticker, stock_var in stock_vars.items():
    std_return = log_returns[ticker]['std_returns']
    skewness = skew(std_return)
    excess_kurt = kurtosis(std_return) - 3
    cornish_fisher_var = stock_var + skewness / 6 * (stock_var ** 2 - 1) + \
                         excess_kurt / 24 * (stock_var ** 3 - 3 * stock_var) - \
                         skewness ** 2 / 36 * (2 * stock_var ** 3 - 5 * stock_var)
    cf_stock_vars[ticker] = cornish_fisher_var
    print(f"{ticker} CF-VaR at {conf_interval}%: {cornish_fisher_var}%")

AAPL CF-VaR at 5%: -0.9117873329342351%
AMZN CF-VaR at 5%: -1.2377948530037945%
TSLA CF-VaR at 5%: -2.053445008439621%
JNJ CF-VaR at 5%: -1.1541723010767764%
COP CF-VaR at 5%: -0.4361577852578613%
KO CF-VaR at 5%: -0.8021261939421721%
JPM CF-VaR at 5%: -2.5958813657572324%
NVDA CF-VaR at 5%: -1.9073278938023068%
UNH CF-VaR at 5%: -2.1219693562344784%
PG CF-VaR at 5%: -2.554544434003965%


### Equal Weighted Portfolio

In [178]:
stock_returns = list(map(lambda x: x['log_returns'], log_returns.values()))
stock_returns = pd.concat(stock_returns, axis = 1)
stock_returns.columns = tickers
portfolio_returns = stock_returns.mean(axis = 1)
portfolio_std_returns = portfolio_returns / portfolio_returns.rolling(252).std().dropna()
portfolio_std_returns.dropna(inplace = True)


In [182]:
model = arch_model(portfolio_returns * 100, vol = 'Garch', p = 1, q = 1)
model_fit = model.fit(disp='off')
omega, alpha, beta = model_fit.params[1:]
next_day_std = (omega + alpha * portfolio_returns.iloc[-1]**2 + beta * portfolio_std_returns.iloc[-1]**2)**0.5
    
# Randomly select 5000 standardized returns from the historical z-scores
random_selection = np.random.choice(portfolio_std_returns, 10000)
    
# Forecasts of the returns
forecasts = next_day_std * random_selection

ptf_var = np.percentile(forecasts, conf_interval)
print(f"Portfolio {conf_interval}% VaR: {var}%")


skewness = skew(std_return)
excess_kurt = kurtosis(std_return) - 3
ptf_cornish_fisher_var = ptf_var + skewness / 6 * (ptf_var ** 2 - 1) + \
                     excess_kurt / 24 * (ptf_var ** 3 - 3 * ptf_var) - \
                     skewness ** 2 / 36 * (2 * ptf_var ** 3 - 5 * ptf_var)

print(f"Portfolio {conf_interval}% CF-VaR: {ptf_cornish_fisher_var}%")

Portfolio 5% VaR: -1.0908711749717688%
Portfolio 5% CF-VaR: -0.9939934805930248%


### Comparison

In [188]:
historical_ptf_var = np.percentile(portfolio_returns, conf_interval)
print(f"VaR under historical data assumption: {historical_ptf_var * 100}%")

mean_ptf_returns = portfolio_returns.mean()
std_ptf_returns = portfolio_returns.std()
num_samples = 10000

samples = np.random.normal(mean_ptf_returns, std_ptf_returns, num_samples)
norm_ptf_var = np.percentile(samples, conf_interval)

print(f"VaR under normal data assumption: {norm_ptf_var * 100}%")



VaR under historical data assumption: -2.141894374907534%
VaR under historical data assumption: -2.3026940459658576%
